# TP 1 - Traitement de données

Dylan Robins - E2I5 - 17 Dec 2021

## Exercice 1: Températures

Dans cet exercice, nous analyserons un jeu de données contenant la température moyenne chaque mois dans 35 villes européennes tout au long d'une année.

In [ ]:
# LIBRARY IMPORT
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from IPython.display import display
sns.set()

In [ ]:
# DATA IMPORT
df = pd.read_excel("datasets/Temperatures.xlsx", sheet_name=0, header=0, index_col=0)

if df.isnull().values.any():
    print("Missing data!")
    exit(1)
else:
    print("No missing data!")

print(f"Dataframe shape: {df.shape[0]} rows, {df.shape[1]} cols")

df

On commence par faire un plot naif de nos données pour voir si on peut en tirer quelquechose. En l'occurence, à part voir que rien ne se détache trop du reste, on n'a pas grand chose d'interprétable:

In [ ]:
df.plot()
plt.xticks(rotation=45)
pass

On va maintenant regarder les moyennes et les variances des différentes colonnes du tableau, ainsi que leurs distributions:

In [ ]:
df.boxplot()
pd.DataFrame({"Moyenne": df.mean(), "Variance": df.var()})

On veut maintenant étudier les intercorrélations entre les différentes colonnes. Le scatter_matrix de base de pandas étant illisible quand on a autant de données, on utilisera un heatmap de seaborn. On constate que toutes les variables sont corrélées positivement, bien que les mois aux opposés des solstices le sont plus que les autres (Janv/Dec, Fev/Nov...).

On calcule ensuite les coefficients de corrélation et de covariance.

In [ ]:
sns.heatmap(df.corr(), vmin=-1, vmax=1, cmap="coolwarm")

print("Covariance:")
display(df.cov())

print("\nCorrélation:")
display(df.corr())

On veut désormais effectuer une ACP afin de réduire la dimension de ces données. On constate que 98,16% de la variance est expliquée par les deux premiers axes: on pourra donc se réduire à une représentation 2D dans la suite.

In [ ]:
acp = PCA()
Xproj = acp.fit_transform(df)

print(
    f"Variance expliquée par les deux premiers vecteur propres:",
    sum(acp.explained_variance_ratio_[:2])
)

print("Somme des variances:", sum(acp.explained_variance_))

labels = [f"V{i}" for i, _ in enumerate(acp.explained_variance_ratio_)]
plt.bar(labels, acp.explained_variance_ratio_, width=0.25, label='Variance ratio')
plt.plot(labels, acp.explained_variance_ratio_.cumsum(), 'r.-', label='Cumulative sum')
plt.title("Diagramme de Pareto")
plt.legend()
pass

On trace maintenant le nuage de points de nos données expliquées par les deux premiers vecteurs propres de notre nouvelle base:

In [ ]:
plt.scatter(Xproj[:,0], Xproj[:,1])
for i, name in enumerate(df.index):
    plt.annotate(name, (Xproj[i,0], Xproj[i,1]), xytext=(2, 2), textcoords='offset points')

On voit donc une différenciation claire entre nos villes, cependant nous ne savons pas à quoi correspondent nos axes, si ce n'est que ce sont des combinaisons linéaires entre les différents mois de l'année. Pour obtenir un peu plus de détail sur cette combinaison linéaire, on trace le cercle de corrélation:

In [ ]:
# Calculate correlations between new data (Xproj) and original columns (df):
corvar = np.zeros((len(df.columns), 2))
for i, col in enumerate(df.columns):
    corvar[i, 0] = np.corrcoef(Xproj[:,0], df.iloc[:, i])[0, 1]
    corvar[i, 1] = np.corrcoef(Xproj[:,1], df.iloc[:, i])[0, 1]
  
cor_df = pd.DataFrame(corvar, columns=["1", "2"], index=df.columns)
print(f"Correlation coefficients:")
display(cor_df)

# Cercle des corrélations
fig, axes = plt.subplots(figsize=(8,8))
axes.set_xlim(-1,1)
axes.set_ylim(-1,1)

# On ajoute les axes
plt.plot([-1,1],[0,0],color='silver',linestyle='-',linewidth=1)
plt.plot([0,0],[-1,1],color='silver',linestyle='-',linewidth=1)
# On ajoute un cercle
cercle = plt.Circle((0,0),1,color='blue',fill=False)
axes.add_artist(cercle)
plt.xlabel("Composante principale 1")
plt.ylabel("Composante principale 2")
plt.title('Cercle des corrélations')
plt.scatter(corvar[:,0],corvar[:,1])
#affichage des étiquettes (noms des variables)
for j, _ in enumerate(df.columns):
  plt.annotate(df.columns[j],(corvar[j,0],corvar[j,1]), xytext=(2, 2), textcoords='offset points')

plt.show()

On peut donc dire que l'axe 1 correspond majoritairement à l'information "il fait chaud dans la ville en général", tandis que l'axe 2 correspond majoritairement à l'information "les hivers sont doux et les étés froids".

Cela explique bien la position des villes données sur le nuage de points précédent:
- Les villes méditerranéennes comme Séville se trouvent à droite,
- Rekyavik, ville chauffée par l'activité volcanique de l'Islande mais se situant proche de l'Arctique, se retrouve tout en haut à gauche de notre graphe.

## Exercice 2: Criminalité

In [ ]:
# DATA IMPORT
df = pd.read_excel("datasets/Criminalite.xlsx", sheet_name=0, header=0, index_col=0)

if df.isnull().values.any():
    print("Missing data!")
    exit(1)
else:
    print("No missing data!")

print(f"Dataframe shape: {df.shape[0]} rows, {df.shape[1]} cols")

Encore une fois, nous avons un jeu de données assez conséquent et opaque à première vue. Le simple graphe des données n'est pas très parlant. Appliquons donc le même raisonnement que précédemment

In [ ]:
ax = df.plot()
plt.xticks(rotation=45)
pass

In [ ]:
df.boxplot()
plt.xticks(rotation=45)
pd.DataFrame({"Moyenne": df.mean(), "Variance": df.var()})

On voit dans le boxplot quelques "outliers" qui peuvent être intéressantes à étudier. On devrait pouvoir les identifier plus clairement une fois le traitement réalisé.

Comme précédemment, on calcule les coefficients de corrélation et on constate que les agressions et les viols sont corrélés positivement tandis que les meurtres et la petite délinquence sont inversement proportionnels.

In [ ]:
sns.heatmap(df.corr(), vmin=-1, vmax=1, cmap="coolwarm")

print("\nCoefficients de corrélation:")
display(df.corr())

### ACP 1: traitement des données centrées

On commence par réaliser un traitement sur les données simplement centrées sur l'origine des axes.

Pour centrer une série de données, on lui retire sa moyenne.

In [ ]:
mean = df.to_numpy().mean(axis=0)
df_centered = df - df.mean()

plt.scatter(df.index, df["Meurtres"])
plt.scatter(df.index, df_centered["Meurtres"])
plt.xticks(rotation=90, fontsize = 8)
pass

Maintenant que nos données sont centrées, on peut procéder à l'ACP. On voit qu'encore une fois, les deux premiers axes nous donnet largement assez d'information pour décrire la variabilité de notre dataset:

In [ ]:
acp = PCA()
Xproj = acp.fit_transform(df_centered)

print(
    f"Variance expliquée par les deux premiers vecteur propres:",
    sum(acp.explained_variance_ratio_[:2])
)

print("Somme des variances:", sum(acp.explained_variance_))

labels = [f"V{i}" for i, _ in enumerate(acp.explained_variance_ratio_)]
plt.bar(labels, acp.explained_variance_ratio_, width=0.25, label='Variance ratio')
plt.plot(labels, acp.explained_variance_ratio_.cumsum(), 'r.-', label='Cumulative sum')
plt.title("Diagramme de Pareto")
plt.legend()
pass

On trace donc le nuage de points selon ces deux axes ainsi que le cercle de corrélation, et on obtient les graphes suivants:

In [ ]:
plt.scatter(Xproj[:,0], Xproj[:,1])
for i, name in enumerate(df_centered.index):
    plt.annotate(name, (Xproj[i,0], Xproj[i,1]), xytext=(2, 2), textcoords='offset points', fontsize=8)

plt.figure()

# Calculate correlations between new data (Xproj) and original columns (df):
corvar = np.zeros((len(df_centered.columns), 2))
for i, col in enumerate(df_centered.columns):
    corvar[i, 0] = np.corrcoef(Xproj[:,0], df_centered.iloc[:, i])[0, 1]
    corvar[i, 1] = np.corrcoef(Xproj[:,1], df_centered.iloc[:, i])[0, 1]
  
# Cercle des corrélations
fig, axes = plt.subplots(figsize=(8,8))
axes.set_xlim(-1,1)
axes.set_ylim(-1,1)

# On ajoute les axes
plt.plot([-1,1],[0,0],color='silver',linestyle='-',linewidth=1)
plt.plot([0,0],[-1,1],color='silver',linestyle='-',linewidth=1)
# On ajoute un cercle
cercle = plt.Circle((0,0),1,color='blue',fill=False)
axes.add_artist(cercle)
plt.xlabel("Composante principale 1")
plt.ylabel("Composante principale 2")
plt.title('Cercle des corrélations')
plt.scatter(corvar[:,0],corvar[:,1])
#affichage des étiquettes (noms des variables)
for j, _ in enumerate(df_centered.columns):
  plt.annotate(df_centered.columns[j],(corvar[j,0],corvar[j,1]), xytext=(2, 2), textcoords='offset points')

plt.show()
pass

On voit donc que l'axe principal correspond en quelque sorte à un taux de criminalité général, tandis que le second établit une distinction sur le nombre de vols à la tire.

On peut donc dire que
- L'Arizona a le plus grand taux de criminalité, le West Virginia le plus faible
- Le Wyoming et l'Iowa ont une proportion de vols à la tire supérieure que le Massachussets.

On remarque cependant qu'il est difficile de distinguer les Etats entre eux pour ce qui est des crimes autre que le vol à la tire étant donné qu'ils sont tous expliqués de façon presque identique par le 2e axe.

### ACP 2: traitement des données centrées / réduites

On veut maintenant faire exactement pareil mais avec un jeu de données centré et réduit.

Pour centrer/réduire nos données, on leur retire leur moyenne puis on les divise par la racine carrée de leur variance

In [ ]:
mean = df.to_numpy().mean(axis=0)
df_cr = (df - df.mean()) / df.std()

plt.scatter(df.index, df["Meurtres"])
plt.scatter(df.index, df_cr["Meurtres"])
plt.xticks(rotation=90, fontsize = 8)
pass

In [ ]:
acp = PCA()
Xproj = acp.fit_transform(df_cr)

print(
    f"Variance expliquée par les deux premiers vecteur propres:",
    sum(acp.explained_variance_ratio_[:2])
)

print("Somme des variances:", sum(acp.explained_variance_))

labels = [f"V{i}" for i, _ in enumerate(acp.explained_variance_ratio_)]
plt.bar(labels, acp.explained_variance_ratio_, width=0.25, label='Variance ratio')
plt.plot(labels, acp.explained_variance_ratio_.cumsum(), 'r.-', label='Cumulative sum')
plt.title("Diagramme de Pareto")
plt.legend()
pass

On voit ici une différence notable par rapport à la méthode précédente: ici les deux premiers axes ne suffisent pas tout à fait à expliquer l'entièreté de la variabilité, ce seuil étant habituellement définit à 80%. Ici nous avons 76% donc cela reste correct, mais ce serait sûrement plus intéressant de faire des représentations en 3 dimentions pour interpréter nos données.

In [ ]:
plt.scatter(Xproj[:,0], Xproj[:,1])
for i, name in enumerate(df_cr.index):
    plt.annotate(name, (Xproj[i,0], Xproj[i,1]), xytext=(2, 2), textcoords='offset points', fontsize=8)

plt.figure()

# Calculate correlations between new data (Xproj) and original columns (df):
corvar = np.zeros((len(df_cr.columns), 2))
for i, col in enumerate(df_centered.columns):
    corvar[i, 0] = np.corrcoef(Xproj[:,0], df_centered.iloc[:, i])[0, 1]
    corvar[i, 1] = np.corrcoef(Xproj[:,1], df_cr.iloc[:, i])[0, 1]
  
# Cercle des corrélations
fig, axes = plt.subplots(figsize=(8,8))
axes.set_xlim(-1,1)
axes.set_ylim(-1,1)

# On ajoute les axes
plt.plot([-1,1],[0,0],color='silver',linestyle='-',linewidth=1)
plt.plot([0,0],[-1,1],color='silver',linestyle='-',linewidth=1)
# On ajoute un cercle
cercle = plt.Circle((0,0),1,color='blue',fill=False)
axes.add_artist(cercle)
plt.xlabel("Composante principale 1")
plt.ylabel("Composante principale 2")
plt.title('Cercle des corrélations')
plt.scatter(corvar[:,0],corvar[:,1])
#affichage des étiquettes (noms des variables)
for j, _ in enumerate(df_cr.columns):
  plt.annotate(df_cr.columns[j],(corvar[j,0],corvar[j,1]), xytext=(2, 2), textcoords='offset points')

plt.show()
plt.scatter(Xproj[:,0], Xproj[:,1])
for i, name in enumerate(df_cr.index):
    plt.annotate(name, (Xproj[i,0], Xproj[i,1]), xytext=(2, 2), textcoords='offset points', fontsize=8)

plt.figure()

# Calculate correlations between new data (Xproj) and original columns (df):
corvar = np.zeros((len(df_cr.columns), 2))
for i, col in enumerate(df_centered.columns):
    corvar[i, 0] = np.corrcoef(Xproj[:,0], df_centered.iloc[:, i])[0, 1]
    corvar[i, 1] = np.corrcoef(Xproj[:,1], df_cr.iloc[:, i])[0, 1]
  
# Cercle des corrélations
fig, axes = plt.subplots(figsize=(8,8))
axes.set_xlim(-1,1)
axes.set_ylim(-1,1)

# On ajoute les axes
plt.plot([-1,1],[0,0],color='silver',linestyle='-',linewidth=1)
plt.plot([0,0],[-1,1],color='silver',linestyle='-',linewidth=1)
# On ajoute un cercle
cercle = plt.Circle((0,0),1,color='blue',fill=False)
axes.add_artist(cercle)
plt.xlabel("Composante principale 1")
plt.ylabel("Composante principale 2")
plt.title('Cercle des corrélations')
plt.scatter(corvar[:,0],corvar[:,1])
#affichage des étiquettes (noms des variables)
for j, _ in enumerate(df_cr.columns):
  plt.annotate(df_cr.columns[j],(corvar[j,0],corvar[j,1]), xytext=(2, 2), textcoords='offset points')

plt.show()
pass

In [ ]:
fig = plt.figure()
ax = fig.gca(projection='3d')

ax.scatter(Xproj[:,0], Xproj[:,1], Xproj[:,2], linewidth=0.2)

plt.show()

for i, name in enumerate(df_cr.index):
    plt.annotate(name, (Xproj[i,0], Xproj[i,1], Xproj[:,2]), xytext=(2, 2, 2), textcoords='offset points', fontsize=8)

On voit que même si seulement 76% de la variabilité est expliquée, ce graphe est beaucoup plus intéressant à interprérer que la version seulement centrée! En effet on a une distinction claire entre les différentes contributions des axes, et on peut voir clairement que l'axe 2 correspond à l'inverse de la gravité des crimes: plus un Etat est bas sur l'axe, plus il y a de meurtres, de viols et d'agressions par opposition à de la petite délinquence.

Comme précédemment, on peut donc dire que l'Arizona a un taux de criminalité supérieur à la moyenne, mais qu'en revanche ces crimes sont moins "graves" que ceux commis au Mississipi.

On voit donc clairement l'intéret de traiter les données centrées/réduites: cela nous permet de mieux distinguer les contributions des différentes variables que l'on souhaite prendre en compte, ce qui facilite grandement l'interprétation des résultats